In [ ]:
from langgraph.graph import StateGraph,START,END,MessagesState
from langchain.pydantic_v1 import BaseModel,Field
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langchain import hub
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores.chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import os

c:\Users\ADMIN\anaconda3\envs\jay\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\ADMIN\anaconda3\envs\jay\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin, urlparse
from langchain.document_loaders import WebBaseLoader

# Function to get all links from a page
def get_all_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = set()
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        # Convert relative URLs to absolute URLs
        full_url = urljoin(url, href)
        links.add(full_url)
    return links

# Function to filter URLs by domain
def filter_by_domain(urls, target_domain):
    filtered_urls = []
    for url in urls:
        parsed_url = urlparse(url)
        if parsed_url.netloc == target_domain:
            filtered_urls.append(url)
        else:
            print(f"Skipping external URL: {url}")
    return filtered_urls

# Start URL
start_url = "https://dahfood.com/"

# Get all links from the start page
all_links = get_all_links(start_url)

# Filter links to include only those from the same domain
target_domain = urlparse(start_url).netloc  # Extract domain (e.g., 'dahfood.com')
valid_links = filter_by_domain(all_links, target_domain)

# Scrape each valid link
# all_data = []
# for link in valid_links:
#     try:
#         loader = WebBaseLoader(link)
#         data = loader.load()
#         all_data.extend(data)
#     except Exception as e:
#         print(f"Failed to scrape {link}: {e}")

# # Print all scraped data
# print(all_data)

Skipping external URL: http://linkedin.com/in/setareh-omidi-6123b6254
Skipping external URL: https://t.me/dateexporters
Skipping external URL: https://t.me/dahfood
Skipping external URL: https://www.facebook.com/setarehomidii/
Skipping external URL: https://www.linkedin.com/in/setarehomidi
Skipping external URL: https://wa.me/989384444008
Skipping external URL: mailto:sales@dahfood.com?Subject=sales
Skipping external URL: https://goo.gl/maps/nfEjMwTy8ke93KCn6
Skipping external URL: tel:+989384444008
Skipping external URL: go:top
Skipping external URL: https://instagram.com/dahfoodcompany?igshid=YmMyMTA2M2Y=
Skipping external URL: tel:989384444008
Skipping external URL: https://www.instagram.com/3pehr_mb/?hl=en
Skipping external URL: mailto:sales@dahfood.com?Subject=footer
Skipping external URL: https://www.facebook.com/setarehomidii


In [12]:
valid_links

['https://dahfood.com/DAHFOOD-Sultana-raisin/',
 'https://dahfood.com/DAHFOOD-Red-pistachio-kernel/',
 'https://dahfood.com/DAHFOOD-Media/',
 'https://dahfood.com/DAHFOOD-Dates-Catalogues/',
 'https://dahfood.com/Red-Apple/',
 'https://dahfood.com/DAHFOOD-Kabkab-dates/',
 'https://dahfood.com/DAHFOOD-Fandoghi/',
 'https://dahfood.com/DAHFOOD-Golden-Raisin/',
 'https://dahfood.com/DAHFOOD-Contact-us/',
 'https://dahfood.com/DAHFOOD-Moringa-leaf/',
 'https://dahfood.com/documents/',
 'https://dahfood.com/Date-seed-kernel/',
 'https://dahfood.com/',
 'https://dahfood.com/DAHFOOD-Moringa-powder/',
 'https://dahfood.com/DAHFOOD-About-us/',
 'https://dahfood.com/Date-syrup/',
 'https://dahfood.com/DAHFOOD-Pistachio-Catalogues/',
 'https://dahfood.com/DAHFOOD-Dried-fruit-Catalogues/',
 'https://dahfood.com/DAHFOOD-Golden-Kashmari-raisin/',
 'https://dahfood.com/DAHFOOD-Sun-dried-raisin/',
 'https://dahfood.com/DAHFOOD-Sliced-dates/',
 'https://dahfood.com/DAHFOOD-Packing-Label/',
 'https://da

In [13]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [14]:
urls = valid_links

In [22]:
doc = [WebBaseLoader(url).load() for url in urls]

print(doc)

doc_split = [item for split_doc in doc for item in split_doc]

print(doc_split)

text_split = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,chunk_overlap=100
)
splited_text = text_split.split_documents(doc_split)

vectorstore = Chroma(
    collection_name="jay_data",
    embedding_function=embedding
)
vectorstore.add_documents(splited_text)

retriever = vectorstore.as_retriever()

[[Document(metadata={'source': 'https://dahfood.com/DAHFOOD-Sultana-raisin/', 'title': 'DAHFOOD | Sultana raisin', 'description': 'sultana raisin is produced from seedless, white grapes. Sultana raisin is famous because it has delicate and unique flavor. This raisin has lighter color and sweeter taste compared to other types of raisins. The color scale ranges from light to dark as well as various size options. For the dipping process, it is dipped in a solution of Potash and oil to accelerate the drying process and make the color of the raisin lighter. ¬ÝIf you are looking for a high quality Sultana raisin, as one of the best iranian raisin producer and suppliers we can help you.\nThe color of Sultana raisins is often from light brown to dark brown. Raisins themselves have a sweet taste that has its unique customers around the world. The main countries that buy Sultana raisins are the European Union, Russia, China, Ukraine, Arab countries, Iraq and Southeast Asia.\n‚Äã‚Äã‚Äã‚Äã‚Äã‚Äã‚Ä

In [23]:
splited_text

[Document(metadata={'source': 'https://dahfood.com/DAHFOOD-Sultana-raisin/', 'title': 'DAHFOOD | Sultana raisin', 'description': 'sultana raisin is produced from seedless, white grapes. Sultana raisin is famous because it has delicate and unique flavor. This raisin has lighter color and sweeter taste compared to other types of raisins. The color scale ranges from light to dark as well as various size options. For the dipping process, it is dipped in a solution of Potash and oil to accelerate the drying process and make the color of the raisin lighter. ¬ÝIf you are looking for a high quality Sultana raisin, as one of the best iranian raisin producer and suppliers we can help you.\nThe color of Sultana raisins is often from light brown to dark brown. Raisins themselves have a sweet taste that has its unique customers around the world. The main countries that buy Sultana raisins are the European Union, Russia, China, Ukraine, Arab countries, Iraq and Southeast Asia.\n‚Äã‚Äã‚Äã‚Äã‚Äã‚Äã‚Äã

In [28]:
question = 'i want to buy product'
document = retriever.get_relevant_documents(question)

In [29]:
prompt = hub.pull('rlm/rag-prompt')

rag_chain = prompt | llm | StrOutputParser()

rag_chain.invoke({'context':document,'question':question})

c:\Users\ADMIN\anaconda3\envs\jay\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'DAHFOOD sells Ahmad Aghaei pistachios, date paste, and date seeds.  They also offer Fandoghi pistachios. Contact information is provided in the text to make a purchase.'